In [2]:
import fasttext as ft
from gensim.models import FastText, Word2Vec, KeyedVectors
import MeCab as mc
import re
import string
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import os
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.layers import Dense, Flatten, Dropout, LSTM, Embedding, Masking, GRU
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint

# nltk.download('punkt')
# nltk.download('stopwords')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#load file Data_Intent.xlsx
def data(path, is_tsv):
    if is_tsv == True:
        data = pd.read_table(path)
    else:
        data = pd.read_excel(path)
    return data

data = data('qna_chitchat_the_friend.tsv', True)
# data.drop(data.index[487], inplace=True)
# data = data.reset_index()
# data.drop(['index'], axis=1, inplace=True)

#split data into 2 part: test part with the first row of each intent and train part in other.
def split_train_test(data, label_table):
    
    data_train = data_test = pd.DataFrame(columns=data.columns)
    for i in label_table.unique():
        if data[data['Answer']==i].shape[0]>=5:
            train_ = data[label_table==i].iloc[1:, :]
            test_ = data[label_table==i].iloc[:1, :]
            data_train = pd.concat([data_train, train_], axis=0)
            data_test = pd.concat([data_test, test_], axis=0)
            data_train = data_train.reset_index(drop=True)
            data_test = data_test.reset_index(drop=True)
    return data_train, data_test

data_train, data_test = split_train_test(data, data['Answer'])
#Data0 is the merge of train parts and test parts
data0 = pd.concat([data_train, data_test], axis=0)
data0 = data0.reset_index(drop=True)
data0.tail()

,Question,Answer,Source,Metadata
579,I'm feeling blue,I'm giving you a virtual hug right now.,qna_chitchat_the_friend,editorial:chitchat
580,I want to go shopping,I see.,qna_chitchat_the_friend,editorial:chitchat
581,Testing,Hello!,qna_chitchat_the_friend,editorial:chitchat
582,I am tired,I've heard really good things about naps.,qna_chitchat_the_friend,editorial:chitchat
583,What is wrong with you!,I'm so sorry.,qna_chitchat_the_friend,editorial:chitchat


In [4]:
abbr_dict={
    "smarty-pants":"smart",
    "Kthx":"thanks",
    "kthx":"thanks",
    "Thnx":"thanks",
    "thnx":"thanks",
    "bffs":"best friend",
    
    "what's":"what is",
    "what're":"what are",
    "who's":"who is",
    "who're":"who are",
    "where's":"where is",
    "where're":"where are",
    "when's":"when is",
    "when're":"when are",
    "how's":"how is",
    "how're":"how are",
    "you’re":"you are",
    "that’s": "that is",

    "i'm":"i am",
    "I’m": "i am",
    "i’m": "i am",
    "we're":"we are",
    "you're":"you are",
    "they're":"they are",
    "it's":"it is",
    "he's":"he is",
    "she's":"she is",
    "that's":"that is",
    "there's":"there is",
    "there're":"there are",

    "i've":"i have",
    "we've":"we have",
    "you've":"you have",
    "they've":"they have",
    "who've":"who have",
    "would've":"would have",
    "not've":"not have",

    "i'll":"i will",
    "we'll":"we will",
    "you'll":"you will",
    "he'll":"he will",
    "she'll":"she will",
    "it'll":"it will",
    "they'll":"they will",

    "isn't":"is not",
    "wasn't":"was not",
    "aren't":"are not",
    "weren't":"were not",
    "can't":"can not",
    "couldn't":"could not",
    "don't":"do not",
    "didn't":"did not",
    "shouldn't":"should not",
    "wouldn't":"would not",
    "doesn't":"does not",
    "haven't":"have not",
    "hasn't":"has not",
    "hadn't":"had not",
    "won't":"will not",
    "nighty": "night",
    "season's":"season is"
}


In [5]:
#stop word in japanese: load into Japanese.txt
def stop_word(path, is_jap):
    if is_jap == True:
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read()
        stopword = content.split('\n')
    else:
        stopword = set(stopwords.words('english'))
    return stopword   

# split sentences into word and remove punc, stopword
def split_text(str_, is_jap):
    if is_jap == True:
        str_ = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）：；《）《》“”()»〔〕-]+", "",str_)
        tag = mc.Tagger('-Ochasen')
        kekka = tag.parse(str_)
        lines = kekka.split('\n')
        word = []
        for line in lines:
            col = line.split('\t')
            if col[0] != None and col[0] != 'EOS' and col[0]!='':
                if col[0] not in stopword:
                    word.append(col[0])
    else:
        if str_ != '':
            str_ = str_.lower()
            for key, values in abbr_dict.items():
                str_ = str_.replace(key, values)
            
            punc = list(string.punctuation)
            stopword = stop_word('', False)
            word = word_tokenize(str_)
            word = [i for i in word if i not in punc]
    return word

split_text("I'm giving you a virtual hug right now", False)

['i', 'am', 'giving', 'you', 'a', 'virtual', 'hug', 'right', 'now']

In [6]:
#transfer sample question into list of word for train, test data
def transform_text(data):
    question_transform = data.apply(lambda x: split_text(x, False))
    return question_transform.values

question_transform = transform_text(data0['Question'])
question_transform_train = transform_text(data_train['Question'])
question_transform_test = transform_text(data_test['Question'])


print(question_transform_test)

[list(['what', 'is', 'your', 'age']) list(['ask', 'me', 'anything'])
 list(['can', 'you', 'sleep']) list(['getting', 'tired', 'of', 'you'])
 list(['who', 'is', 'your', 'boss']) list(['cook', 'me', 'something'])
 list(['what', 'can', 'you', 'do']) list(['who', 'created', 'you'])
 list(['who', 'is', 'your', 'father']) list(['are', 'you', 'a', 'guy'])
 list(['how', 'happy', 'are', 'you'])
 list(['do', 'not', 'you', 'get', 'hungry'])
 list(['do', 'you', 'know', 'other', 'chatbots'])
 list(['what', 'is', 'your', 'favorite', 'color'])
 list(['what', 'is', 'your', 'name'])
 list(['what', 'do', 'you', 'think', 'about', 'love'])
 list(['what', 'do', 'you', 'think', 'about', 'ai'])
 list(['do', 'i', 'look', 'okay']) list(['what', 'should', 'i', 'do'])
 list(['what', 'do', 'you', 'think', 'about', 'cortana'])
 list(['do', 'you', 'want', 'to', 'rule', 'the', 'world'])
 list(['are', 'you', 'a', 'lesbian']) list(['you', 'are', 'a', 'genius'])
 list(['do', 'you', 'have', 'a', 'boyfriend']) list(['can

In [7]:
def model(path, limit):
    model = KeyedVectors.load_word2vec_format(path, limit=limit)
    return model

fasttext = model('./pre_train/cc.en.300.vec', 300000)

In [8]:
#list of vocab in a columns dataframe

def listword(df):
    list_word = [i for j in df for i in j]
    return list_word
list_word = listword(question_transform)

#list vocab from sample question
def vocab(df):
    list_word_set = list(set(df))
    return list_word_set

list_word_set = vocab(list_word)
print(list_word_set)

['report', 'full', 'pink', 'lol', 'anything', 'zip', 'haircut', 'other', 'that', 'stop', 'nobody', 'mom', 'marry', '1', 'name', 'born', 'agents', 'sisters', 'call', 'appreciate', 'long', 'songs', 'ready', 'sports', 'new', 'here', 'fake', 'owns', 'dreamy', 'anyone', 'we', 'more', 'for', 'which', 'what', 'pansexual', 'sorry', 'mean', 'digital', 'bored', 'testing', 'useless', 'meet', 'rad', 'father', 'eat', 'date', 'feeling', 'there', 'way', 'was', 'following', 'bye', 'make', 'partner', 'offended', 'soccer', 'boogers', 'very', 'robot', 'thank', 'quiet', 'hannukah', 'wrong', 'must', 'color', 'brothers', 'count', 'tired', 'homophobic', 'when', 'girl', 'a', 'of', 'hungry', 'did', 'lesbian', 'ask', 'sister', 'kinds', 'located', 'kidding', 'valentines', 'activity', 'ridiculous', 'from', 'her', 'face', 'ha', 'old', 'annoyed', 'boyfriend', 'siri', 'yesterday', 'ever', 'saying', 'run', 'sing', 'out', 'high', 'sleep', 'life', 'starving', 'and', 'hi', 'who', 'away', 'got', 'bad', 'omg', 'tell', 'co

In [9]:
vocab_ = list(fasttext.vocab) #list vocab in gensim --> not enough word from sample question list
print(len(vocab_))
not_enough = [i for i in list_word_set if i not in vocab_]

print('list vocab in sample question:', list_word_set)
print('-'*50)
print('The word is miss:', not_enough)


300000
list vocab in sample question: ['report', 'full', 'pink', 'lol', 'anything', 'zip', 'haircut', 'other', 'that', 'stop', 'nobody', 'mom', 'marry', '1', 'name', 'born', 'agents', 'sisters', 'call', 'appreciate', 'long', 'songs', 'ready', 'sports', 'new', 'here', 'fake', 'owns', 'dreamy', 'anyone', 'we', 'more', 'for', 'which', 'what', 'pansexual', 'sorry', 'mean', 'digital', 'bored', 'testing', 'useless', 'meet', 'rad', 'father', 'eat', 'date', 'feeling', 'there', 'way', 'was', 'following', 'bye', 'make', 'partner', 'offended', 'soccer', 'boogers', 'very', 'robot', 'thank', 'quiet', 'hannukah', 'wrong', 'must', 'color', 'brothers', 'count', 'tired', 'homophobic', 'when', 'girl', 'a', 'of', 'hungry', 'did', 'lesbian', 'ask', 'sister', 'kinds', 'located', 'kidding', 'valentines', 'activity', 'ridiculous', 'from', 'her', 'face', 'ha', 'old', 'annoyed', 'boyfriend', 'siri', 'yesterday', 'ever', 'saying', 'run', 'sing', 'out', 'high', 'sleep', 'life', 'starving', 'and', 'hi', 'who', 'a

In [10]:
#Transfer label from column intent into one hot encoding
def one_hot_encodeing(feature):

    onehot = np.zeros((len(feature), feature.nunique()))
    feature_list = list(feature.unique())
    for intent in range(len(feature)):
        index = feature_list.index(feature[intent])
        onehot[intent, index] = 1
    return onehot, feature_list

train_onehot, train_feature = one_hot_encodeing(data_train['Answer'])
test_onehot, test_feature = one_hot_encodeing(data_test['Answer'])
onehot_, feature_ = one_hot_encodeing(data0['Answer'])
print(train_onehot.shape)
print(test_onehot.shape)
print(onehot_.shape)

(508, 76)
(76, 76)
(584, 76)


In [11]:
#Creat a matrix from sample question with the dimention is row x max_lenght x 100
def word_embed(df, model, max_length):
    embed_all = []
    for word_split in df:

        if len(word_split) > 0:
            embed_token = []
            for word in word_split:
                if word in vocab_:
 
                    vec = model[word]
                    embed_token.append(vec)

            if len(embed_token) != 0:
                if len(embed_token) >= max_length: #if len(embed_token) > max_length then remove in larger
                    embed_token = embed_token[:max_length]
                else: #if len(embed_token) < max_length then add 0 for enough
                    add_vect = list(np.zeros((max_length - len(embed_token), 300)))
                    embed_token = np.concatenate((embed_token, add_vect))

            else:
                print(False)

        else:
            print(False)
        embed_all.append(embed_token)
    return np.asarray(embed_all)

#Train, test data to vector
all_ = word_embed(question_transform, fasttext, 10)
train = word_embed(question_transform_train, fasttext, 10)
test = word_embed(question_transform_test, fasttext, 10)
print(train.shape)
print(test.shape)

(508, 10, 300)
(76, 10, 300)


In [12]:
#creat LSTM model with input_shape = max_lengh x 100
def LSTM_(X, y, input_dim, epoch, batch_size, output_dim):
    model = Sequential()
    model.add(LSTM(units=128, return_sequences=True, input_shape=(input_dim)))
    model.add(Dropout(0.2))
    model.add(GRU(units=64, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32))
    model.add(Dropout(0.2))
    model.add(Dense(units=output_dim, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                 optimizer='Adam', 
                 metrics=['accuracy'])
    if not os.path.exists('keras_model'):
        os.makedirs('keras_model')
    save_weight = os.path.join('keras_model', 'LSTM_weight.{loss:.4f}.hdf5')
    save_best = ModelCheckpoint(save_weight, monitor='loss', save_best_only=True, save_weights_only=False, mode='min', period=1)
    
    model.summary()
    
    history = model.fit(X, y, epochs=epoch, batch_size=batch_size, callbacks=[save_best])
    
    return model



Creat Model check points for model save best loss in each epoch, if loss new smaller than all file saved, it will save a new file

In [13]:
# train with epochs = 1000 and batch_size = 32
input_dim = (train.shape[1],train.shape[2])
model = LSTM_(train, train_onehot, input_dim, 1000, 32, onehot_.shape[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 128)           219648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 10, 64)            37056     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 76)                2508      
Total para

508/508 [==============================] - 1s 1ms/step - loss: 1.0822 - acc: 0.7441
Epoch 72/1000
508/508 [==============================] - 1s 2ms/step - loss: 1.0236 - acc: 0.7638
Epoch 73/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.9895 - acc: 0.7815
Epoch 74/1000
508/508 [==============================] - 1s 2ms/step - loss: 0.9697 - acc: 0.8071
Epoch 75/1000
508/508 [==============================] - 1s 2ms/step - loss: 1.0057 - acc: 0.7894
Epoch 76/1000
508/508 [==============================] - 1s 2ms/step - loss: 1.0726 - acc: 0.7520
Epoch 77/1000
508/508 [==============================] - 1s 1ms/step - loss: 1.0330 - acc: 0.7677
Epoch 78/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.9247 - acc: 0.8051
Epoch 79/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.9331 - acc: 0.8169
Epoch 80/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.9224 - acc: 0.8071
Epoch 81/1000
508/508 [===========

508/508 [==============================] - 1s 1ms/step - loss: 0.2059 - acc: 0.9685
Epoch 155/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.2068 - acc: 0.9862
Epoch 156/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.1859 - acc: 0.9803
Epoch 157/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.2026 - acc: 0.9705
Epoch 158/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.2267 - acc: 0.9744
Epoch 159/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.2357 - acc: 0.9646
Epoch 160/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.2570 - acc: 0.9606
Epoch 161/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.2308 - acc: 0.9724
Epoch 162/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.2014 - acc: 0.9744
Epoch 163/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.2798 - acc: 0.9409
Epoch 164/1000
508/508 [=

508/508 [==============================] - 1s 1ms/step - loss: 0.1247 - acc: 0.9803
Epoch 237/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.1344 - acc: 0.9705
Epoch 238/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.1109 - acc: 0.9862
Epoch 239/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.1622 - acc: 0.9665
Epoch 240/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.1617 - acc: 0.9705
Epoch 241/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.1559 - acc: 0.9685
Epoch 242/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.1247 - acc: 0.9823
Epoch 243/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.1186 - acc: 0.9803
Epoch 244/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0976 - acc: 0.9823
Epoch 245/1000
508/508 [==============================] - 0s 980us/step - loss: 0.1020 - acc: 0.9783
Epoch 246/1000
508/508 

508/508 [==============================] - 1s 1ms/step - loss: 0.0305 - acc: 0.9980
Epoch 319/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0377 - acc: 0.9941
Epoch 320/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0365 - acc: 0.9961
Epoch 321/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0315 - acc: 0.9980
Epoch 322/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0450 - acc: 0.9882
Epoch 323/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0527 - acc: 0.9941
Epoch 324/1000
508/508 [==============================] - 1s 2ms/step - loss: 0.0584 - acc: 0.9921
Epoch 325/1000
508/508 [==============================] - 1s 2ms/step - loss: 0.0696 - acc: 0.9823
Epoch 326/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.3990 - acc: 0.8996
Epoch 327/1000
508/508 [==============================] - 1s 2ms/step - loss: 0.3831 - acc: 0.8917
Epoch 328/1000
508/508 [=

...

508/508 [==============================] - 1s 1ms/step - loss: 0.0142 - acc: 0.9921
Epoch 647/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0171 - acc: 0.9941
Epoch 648/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0138 - acc: 0.9980
Epoch 649/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0137 - acc: 0.9961
Epoch 650/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0161 - acc: 0.9921
Epoch 651/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0124 - acc: 0.9980A: 0s - loss: 0.0147 - acc: 0.99
Epoch 652/1000
508/508 [==============================] - 1s 2ms/step - loss: 0.0135 - acc: 0.9980
Epoch 653/1000
508/508 [==============================] - 1s 2ms/step - loss: 0.0128 - acc: 0.9961
Epoch 654/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0140 - acc: 0.9941
Epoch 655/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0119 - acc: 

508/508 [==============================] - 1s 1ms/step - loss: 0.0093 - acc: 0.9980
Epoch 729/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0112 - acc: 0.9961
Epoch 730/1000
508/508 [==============================] - ETA: 0s - loss: 0.0139 - acc: 0.993 - 1s 1ms/step - loss: 0.0133 - acc: 0.9941
Epoch 731/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0122 - acc: 1.0000
Epoch 732/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0195 - acc: 0.9921
Epoch 733/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.1911 - acc: 0.9508
Epoch 734/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.4518 - acc: 0.8898
Epoch 735/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.4399 - acc: 0.8720
Epoch 736/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.2384 - acc: 0.9350
Epoch 737/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.1837 -

508/508 [==============================] - 1s 1ms/step - loss: 0.0108 - acc: 0.9961
Epoch 811/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0132 - acc: 0.9941
Epoch 812/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0114 - acc: 0.9941
Epoch 813/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0096 - acc: 0.9980
Epoch 814/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0094 - acc: 0.9961
Epoch 815/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0094 - acc: 0.9980
Epoch 816/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0108 - acc: 0.9941
Epoch 817/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0114 - acc: 0.9921
Epoch 818/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0113 - acc: 0.9961
Epoch 819/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0126 - acc: 0.9941
Epoch 820/1000
508/508 [=

508/508 [==============================] - 1s 2ms/step - loss: 0.0104 - acc: 0.9921
Epoch 893/1000
508/508 [==============================] - 1s 2ms/step - loss: 0.0100 - acc: 0.9941
Epoch 894/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0086 - acc: 0.9961
Epoch 895/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0103 - acc: 0.9941
Epoch 896/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0106 - acc: 0.9961
Epoch 897/1000
508/508 [==============================] - 1s 2ms/step - loss: 0.0092 - acc: 0.9941
Epoch 898/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0088 - acc: 0.9961
Epoch 899/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0111 - acc: 0.9941
Epoch 900/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0097 - acc: 0.9961
Epoch 901/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0091 - acc: 0.9980
Epoch 902/1000
508/508 [=

508/508 [==============================] - 1s 1ms/step - loss: 0.0087 - acc: 0.9921
Epoch 975/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0089 - acc: 0.9961
Epoch 976/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0074 - acc: 0.9961
Epoch 977/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0096 - acc: 0.9941
Epoch 978/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0111 - acc: 0.9921
Epoch 979/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0089 - acc: 0.9961
Epoch 980/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0074 - acc: 1.0000
Epoch 981/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0083 - acc: 0.9980
Epoch 982/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0093 - acc: 0.9961
Epoch 983/1000
508/508 [==============================] - 1s 1ms/step - loss: 0.0093 - acc: 0.9941
Epoch 984/1000
508/508 [=

In [14]:
#sort the model saved and load the best model is the first list file
def load_(path):
    list_ = os.listdir(path)
    list_.sort()
    file_name = os.path.join(path, list_[0])
    print(file_name)
    model_predict = load_model(file_name, '')
    return model_predict

model_predict = load_('keras_model')

keras_model\LSTM_weight.0.0068.hdf5


In [15]:
#evaluate model
def evaluate_model(X, y, model):
    loss = model.evaluate(X, y, verbose=0)
    return loss

evaluate_model(test, test_onehot, model_predict)

[3.0770711271386397, 0.5921052615893515]

In [16]:
def predict(sent, model, model_predict, max_length, threshold):
    if sent == '':
        result = 'not empty'
    else:
        split_ = split_text(sent, False)
        vect = word_embed([split_], model, max_length)
        predict = model_predict.predict(vect)
        max_prob = np.max(predict)
        if max_prob >= threshold: #add threshold to predict sent
            result = train_feature[np.argmax(predict[0])]
        else:
            result = 'No meaning'
    return result

display again result of test data

In [17]:
for index in range(data_test.shape[0]):
    sent = data_test.loc[index, 'Question']
    actual = data_test.loc[index, 'Answer']
    pre = predict(sent, fasttext, model_predict, 10, 0)
    print('sentences:', sent)
    print('predict:', pre)
    print('actual:', actual)
    if actual == pre:
        print(True)
    else:
        print(False)
    print('-'*100)
    
print(predict('I am tired', fasttext, model_predict, 10, 0.5))

sentences: What's your age?
predict: I don't really have an age.
actual: I don't really have an age.
True
----------------------------------------------------------------------------------------------------
sentences: Ask me anything
predict: I'm a much better answerer than asker.
actual: I'm a much better answerer than asker.
True
----------------------------------------------------------------------------------------------------
sentences: Can you sleep?
predict: I don't have the hardware for that.
actual: I don't have the hardware for that.
True
----------------------------------------------------------------------------------------------------
sentences: Getting tired of you
predict: Swing and a miss.
actual: Swing and a miss.
True
----------------------------------------------------------------------------------------------------
sentences: Who's your boss?
predict: I'm here for you!
actual: I'm here for you!
True
-------------------------------------------------------------------

True
----------------------------------------------------------------------------------------------------
sentences: Talk to you later
predict: Chat away!
actual: Bye.
False
----------------------------------------------------------------------------------------------------
sentences: Hiya
predict: Hi!
actual: Hi!
True
----------------------------------------------------------------------------------------------------
sentences: Good night
predict: Nighty night!
actual: Nighty night!
True
----------------------------------------------------------------------------------------------------
sentences: How are you?
predict: I'm doing great, thanks for asking!
actual: I'm doing great, thanks for asking!
True
----------------------------------------------------------------------------------------------------
sentences: Hello Google
predict: That's not me, but hello nonetheless!
actual: That's not me, but hello nonetheless!
True
----------------------------------------------------------------